### **Required Dependencies**

In [1]:
# !pip install googletrans==3.1.0a0
# !pip install rake-nltk

In [2]:
import json, nltk, string, pandas as pd, os, re
from rake_nltk import Rake
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
from googletrans import Translator as tns

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### **Language Transformation**

In [4]:
def lang_transform(query) :
    translator = tns()
    return translator.translate(query, dest="en").text

### **Key Term Extraction**

In [5]:
# stop_words = stopwords.words("english")
# lemmatizer = WordNetLemmatizer()
# key_terms = [lemmatizer.lemmatize(t) for t in transformed_query.split() if t not in stop_words and t not in string.punctuation and len(t) > 2]
# print("Query : ", transformed_query, "\nKey Terms : ", key_terms)

def search(query) :
    transformed_query = lang_transform(query)

    rake = Rake()
    rake.extract_keywords_from_text(transformed_query)
    keywords_extracted = rake.get_ranked_phrases()
    
    return keywords_extracted, transformed_query


### **Data Munging & Visualization**

In [6]:
file = os.getcwd()+"\\first-aid_dataset\\dataset.json"

dataset = json.load(open(file))

# dataset = None
# with open(file, 'r') as f:
#     dataset = json.loads(f.read())

In [7]:
dataframe = pd.DataFrame.from_dict(dataset["intents"])

In [8]:
dataframe.drop(columns="context_set", inplace=True) # Empty Useless
# dataframe.rename(columns = {"tag":"symptoms"}, inplace=True)

In [9]:
# dataframe.shape

In [10]:
# dataframe.columns

In [11]:
# dataframe.info()

In [12]:
dataframe.tag = [str(n).lower() for n in dataframe.tag]
dataframe.patterns = [str(n).lower() for n in dataframe.patterns]

In [13]:
# dataframe

In [14]:
# Empty Checks
cnt = 0
for r in dataframe["responses"].values :
    if " " in r : cnt += 1
        
# print("Empty Rows \t: ", cnt, "\nNon-Empty Rows  : ",len(dataframe.responses)-cnt,"\nTotal Rows \t: ",len(dataframe.responses))

In [15]:
tags = list(dataframe.tag)
patterns = list(dataframe.patterns)
#tags

In [16]:
def search_results(query) :
    d = dict()
    keywords, transformed_query = search(query)
    global tags, patterns, dataframe
    
    for t, p in zip(tags, patterns) :
        for kwrd in keywords :
            if re.findall(kwrd, t) or re.findall(t,kwrd) :
                d[t] = {"question" : dataframe[dataframe.tag == t].patterns.tolist(), "response" : dataframe[dataframe.tag == t].responses.tolist(), "transformed_query" : transformed_query}
    
    if d.keys() : return json.dumps(d)
    else : return "Sorry, I didn't understood what you are saying.!"


In [19]:
query = input("Search : ")
search_results(query)

# search_results("mane mathu dukhe sathe khasi bhi che")

Search : mane daajiyu che


'{"sun burn": {"question": ["[\'how do you treat sun burn?\', \'what to do if i get a sun burn?\', \'which medicine to take if i get sun burn?\', \'how to cure a sun burn?\']"], "response": [["1) Get out of the sun and preferably go indoors. 2) Avoid further sun exposure until the sunburn heals. 3) Cool the skin with a damp cloth or towel, or take a cool bath. 4) Apply moisturizer or aftersun cream, such as aloe vera. 5) Take over-the-counter medications, such as ibuprofen or other nonsteroidal anti-inflammatory medications (NSAIDs) to relieve pain and reduce swelling. 6) Apply hydrocortisone cream to reduce inflammation and itching. 7) Drink extra water to help prevent dehydration. 8) Avoid touching or breaking small blisters to reduce the risk of infection. Blisters protect the skin. 9) Avoid scratching, picking at, or removing peeling skin. 10) Wear loose-fitting clothing while the skin heals. 11) If you experience any difficulty like fever over 103 F (39.4 C) with vomiting or any s

In [ ]:
import pickle

pickle.dumps(search_result(query))